# Text Mine Twitter - NFL Search Patterns


Ryan Timbrook (RTIMBROO)  
DATE:11/30/2019 <br>
Topic: Search Twitter for tweets on specific NFL Players, Coaches, and Teams


## 1. Objective
_____________________________________________________________________________________________
Capture popular opinion of peoples tweets on certain NFL characters. 
Create a corpus of tweets for sentiment analysis


______________________________________________________________________________________________
### Coding Environment Setup
Import packages

In [ ]:
# import packages for analysis and modeling
import pandas as pd #data frame operations
import numpy as np #arrays and math functions
import requests
import os
import io
import pickle
import re
import sys
import string
from os import path
from datetime import date
from datetime import time
from datetime import datetime

In [ ]:
# packages for twitter
import tweepy as tw
from tweepy import OAuthHandler
import json
from tweepy import Stream
from tweepy.streaming import StreamListener

# Twython packages for twitter
from twython import Twython

# packages for NLTK
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer

In [ ]:
# custome python packages
import rtimbroo_utils as br             # custome python helper functions

In [ ]:
# set global properties
notebook_file_name = 'search_twitter_nfl_player'
report_file_name = 'search_twitter_nfl_player'
app_name = 'search_twitter_nfl_player'
log_level = 10 # 10-DEBUG, 20-INFO, 30-WARNING, 40-ERROR, 50-CRITICAL

# setup working directory structure
# set global properties
dataDir = './data'
outputDir = './output'
configDir = './config'
logOutDir = './logs'
imageDir = './images'
modelDir = './models'
corpusDir = './corpus'
# create base output directories if they don't exist
if not os.path.exists(outputDir): os.mkdir(outputDir)
if not os.path.exists(logOutDir): os.mkdir(logOutDir)
if not os.path.exists(imageDir): os.mkdir(imageDir)
if not os.path.exists(modelDir): os.mkdir(modelDir)
if not os.path.exists(dataDir): os.mkdir(dataDir)
if not os.path.exists(configDir): os.mkdir(configDir)
if not os.path.exists(corpusDir): os.mkdir(corpusDir)

In [ ]:
# get a logger for troubleshooting / data exploration
logger = br.getFileLogger(logOutDir+'/',app_name,level=log_level)

In [ ]:
# get current date
now = datetime.utcnow().isoformat()
collection_date = re.findall('^[0-9]{4}-[0-9]{2}-[0-9]{2}',now)
collection_date = collection_date[0]
collection_date

## 2. OBTAIN the data   
________________________________________________________________________________________________
Import external datasets for evaluation

#### Twitter Search API Limits:
[Sandbox Package](https://developer.twitter.com/en/pricing/search-fullarchive)

* Time frame:	Full history
* Tweets per request:	100
* Counts vs. data:	Data only
* Query length:	128 characters
* Operator availability:	Standard
* Rate limit per minute:	30 requests/min
* Enrichments:	n/a
* Dev environments:	1	
* Monthly Tweet cap:	5k	
* Rate limit per second: 10 requests/sec

### Instantiate Twitter API Object
Using Twython 3.6 Twitter API Wrapper
[Twython 3.6.0 reference documentation](https://twython.readthedocs.io/en/latest/api.html)

In [ ]:
# load twitter credentials
with open(f'{configDir}/twitter_credentials.json', 'r') as f:
    tw_cred = json.load(f)

# setup client header arguments to pass along to the API
client_args = {
    'headers':{
        'User-Agent': 'AI_Public_Sentiment_16860838'
    },
    'timeout':300,
    
}
    
# instantiate object
py_tweets = Twython(tw_cred['CONSUMER_KEY'],
                    tw_cred['CONSUMER_SECRET'],
                    tw_cred['ACCESS_TOKEN'],
                    tw_cred['ACCESS_SECRET'],
                   client_args=client_args)

logger.debug(f'{py_tweets.verify_credentials()}')
logger.debug(f'{py_tweets.get_home_timeline()}')

### Configure Search Terms

In [ ]:
# set what to search on
nfl_type = 'player'
search_on = 'deshaun_watson'

# setup base twitter search query
search_terms="deshaun watson"

# add filters to search criteria
filtered_search_terms = search_terms + " -filter:retweets"

#search_start_date = '2019-11-23' # limits to the last 7 days

# number of tweets to return
num_tweets = 100 # sandbox rate limit - 100 tweets per request

In [ ]:
# search dates (from_date, to_date) - Sunday to Sunday
# --- Complete list of the 17 week NFL schedule --- #
search_date_ranges = [
    #('2019-9-1','2019-9-8'),      # week 1
    #('2019-9-8','2019-9-15'),     # week 2
    #('2019-9-15','2019-9-22'),    # week 3
    #('2019-9-22','2019-9-29'),    # week 4
    #('2019-9-29','2019-10-6'),    # week 5
    ('2019-10-6','2019-10-13'),   # week 6
    ('2019-10-13','2019-10-20'),  # week 7
    ('2019-10-20','2019-10-27'),  # week 8
    ('2019-10-27','2019-11-3'),   # week 9
    ('2019-11-3','2019-11-10'),   # week 10
    ('2019-11-10','2019-11-17'),  # week 11
    ('2019-11-17','2019-11-24'),  # week 12
    ('2019-11-24','2019-12-1'),   # week 13
    #('2019-12-1','2019-12-8'),    # week 14
    #('2019-12-8','2019-12-15'),   # week 15
    #('2019-12-15','2019-12-22'),  # week 16
    #('2019-12-22','2019-12-29'),  # week 17
                ]

In [ ]:
def convert_str_date(str_date):
    import time
    day_of_week = str_date.split(' ')[0]
    month = str_date.split(' ')[1]
    day_of_month = str_date.split(' ')[2]
    year = str_date.split(' ')[-1]
    time_of_day = str_date.split(' ')[3]


    new_str_date = f'{month} {day_of_month}, {year}'
    ts = time.strptime(new_str_date, '%b %d, %Y')
    new_ds_str = f'{ts.tm_year}-{ts.tm_mon}-{ts.tm_mday}'

    return new_ds_str, time_of_day, str(ts.tm_year), str(ts.tm_mon), str(ts.tm_mday), str(ts.tm_wday)
    

In [ ]:
# test convert_str_date function - see how output is formatted
#convert_str_date('Sat Nov 23 21:24:19 +0000 2019')

In [ ]:
'''
Function Description: 
'''
def config_query(search_term,since=None,until=None,count=100,lang='en',result_type='mixed'):
    
    # query
    search = {
        'q':search_term,
        'since':since,              # from_date
        'until':until,              # Date format YYYY-MM-DD - returns tweets created before the given date
        'lang':lang,
        'result_type':result_type,    # mixed, recent, popular
        'count':count,                # max is 100, defult is 15 per page

        #'since_id': ,              # returns results with an ID more recent than the specified ID - if the limit of Tweets has occured since the since_id, the since_id will be forced to the oldest ID available
        #'max_id': ,                # returns results with an ID older than or equal to the specified ID
    }
    
    logger.debug(f'config_query: search:\n{search}')
    return search

In [ ]:
'''
Function: Cleans tweet text of URLs, HashTags and @Tags
Returns: Cleaned text, lists of URLs, HasTags and @Tags
'''
def clean_tweet_text_meta(text):
    
    cleaned_text = []
    new_text = None
    urls = []
    hash_tags = []
    at_tags = []
    
    re_hash = re.compile('^#..+')
    re_url = re.compile('^http*')
    re_at = re.compile('^@.+')
    punc_transtable = str.maketrans('', '', string.punctuation)
    
    # loop over tokens collecting meta info and cleaning text
    if len(text) > 0:
        tokens = text.split(' ')
        logger.info(f'tokens: {tokens}')
        
        for tok in tokens:
            
            try:
                # match Hash Tags
                if re.match(re_hash,tok):
                    hash_tags.append(tok.translate(punc_transtable))
                    # hashtags have meaning in tweet text. remove the # character, keep the rest of the text
                    cleaned_text.append(tok[1:])
                # match URLs
                elif re.match(re_url, tok):
                    urls.append(tok)
                # match @Tags
                elif re.match(re_at,tok):
                    at_tags.append(tok)
                # keep text in original format for later cleaning techniques
                else:
                    cleaned_text.append(tok)
                
            except BaseException as be:
                logger.warning(f'clean_tweet_text_meta: ***WARNING*** Caught BaseException: {be}')
            
    else:
        logger.warning(f'clean_tweet_text_meta: ***WARNING***: text is empty: {text}')
    
    logger.info(f'clean_tweet_text_meta: cleaned tokens: {cleaned_text}')
    # join cleaned text back together
    new_text = ' '.join(cleaned_text)
    
    return new_text, urls, hash_tags, at_tags

In [ ]:
'''
Function Description: 
'''
def page_search(twitter,query,text_file,raw_file):
    results = twitter.cursor(twitter.search,**query, return_pages=True)
    # search tweets
    tweets_dict = {'id':[],'created_at':[],'date':[],'time':[],'user':[],'text':[],'favorite_count':[], 'year':[], 'month':[], 'day_of_month':[], 'day_of_week':[]}
    tweets_text_metadata_dict = {'id':[],'date':[],'user':[],'urls':[],'hash_tags':[],'at_tags':[]}
    page_cnt = 0
    result_cnt = 0
    with io.open(f'{text_file}', 'a',encoding='utf8') as f:
        with io.open(f'{raw_file}','a',encoding='utf8') as r:
            try: 
                # page is a list of twitter results
                for i, page in enumerate(results):
                    page_cnt +=1
                    logger.info(f'Page: [{i}]')
                    try:
                        for j,result in enumerate(page):
                            #logger.info(f'result type:{type(result)}')
                            #break
                            
                            result_cnt += 1
                            logger.info(f'Result: [{j}]')
                            try:
                                try:
                                    logger.debug(f'{result["id_str"]} | {result["user"]["screen_name"]} | {result["created_at"]} | {result["text"]} | {result["user"]["favourites_count"]}')
                                except BaseException as be:
                                    logger.warning(f'page_search: ***WARNING***: Caught BaseException writing debug log file: {be}')
                                
                                # dump raw tweet to file as json
                                #raw_tweet = json.load(result)
                                dump = json.dumps(result)
                                r.write(dump)
                                r.write('\n')
                                
                                # dump tweet text to file
                                f.write(f'{result["id_str"]} {result["text"]}')
                                f.write('\n')
                                
                                # add key attributes to tweets dictionary as return results
                                tweets_dict['id'].append(result["id_str"])
                                tweets_dict['created_at'].append(result["created_at"])
                                tweets_dict['favorite_count'].append(result["user"]["favourites_count"])
                                        
                                # call function to parse string date
                                date_time = convert_str_date(result["created_at"]) # get datetime components
                                
                                tweets_dict['date'].append(date_time[0])
                                tweets_dict['time'].append(date_time[1])        
                                tweets_dict['user'].append(result["user"]["screen_name"])
                                
                                # call function to parse text for metadata
                                clean_text = clean_tweet_text_meta(result["text"])
                        
                                tweets_dict['text'].append(clean_text[0])
                                        
                                # create dictionary of tweet text metadata
                                tweets_text_metadata_dict['id'].append(result["id_str"])
                                tweets_text_metadata_dict['date'].append(date_time[0])
                                tweets_text_metadata_dict['user'].append(result["user"]["screen_name"])
                                tweets_text_metadata_dict['urls'].append(clean_text[1])
                                tweets_text_metadata_dict['hash_tags'].append(clean_text[2])
                                tweets_text_metadata_dict['at_tags'].append(clean_text[3])
                                
                                # track timeseries attributes for granular reporting and visualizations
                                tweets_dict['year'].append(date_time[2])
                                tweets_dict['month'].append(date_time[3])
                                tweets_dict['day_of_month'].append(date_time[4])
                                tweets_dict['day_of_week'].append(date_time[5])
                                       
                                #break

                            except BaseException as be:
                                logger.warning(f'**WARNING** Caught BaseException: {be}')
                                             
                    except BaseException as be:
                        logger.warning(f'**WARNING** Caught BaseException: {be}')
                    #break
            except BaseException as be:
                logger.warning(f'**WARNING** Caught BaseException: {be}')
    
    logger.info(f'page_search: processed page_cnt:[{page_cnt}] | total result_cnt: [{result_cnt}]')
    
    return pd.DataFrame.from_dict(tweets_dict), pd.DataFrame.from_dict(tweets_text_metadata_dict)
    

## Execute Twitter Search

[Twython 3.6.0 reference documentation](https://twython.readthedocs.io/en/latest/api.html)

In [ ]:
# sets the search iteration for directory creation - this way we won't overwrite potentially useful data already created
# set this once, then comment out
search_iteration = 1

In [ ]:
'''
Execute Twitter search by pre-configured date ranges
'''
search_iteration +=1 # each time this block of code is ran, the search iteration will update and create a new output directory structure
search_range_results_df = pd.DataFrame()
search_tweets_text_meta_df = pd.DataFrame()

# execute search by date ranges
for dates in search_date_ranges:
    search_range = f'{dates[0]}_{dates[1]}'
    logger.info(f'search_range: {search_range}')
    
    # output file names based on date range search
    outputPath = f'{dataDir}/{nfl_type}/{search_on}/v{search_iteration}/{search_range}'
    if not os.path.exists(outputPath): os.makedirs(outputPath)
        
    tweet_filename=f'{outputPath}/tweet_text.txt'
    raw_filename=f'{outputPath}/tweet_raw.txt'
        
    if not os.path.exists(f'{tweet_filename}'): open(f'{tweet_filename}', 'a').close()
    if not os.path.exists(f'{raw_filename}'): open(f'{raw_filename}', 'a').close()
    
    
    # configure query by dates
    query = config_query(filtered_search_terms,since=dates[0],until=dates[1],count=2)
    #break
    
    #------- EXECUTES TWITTER SEARCH -------------------#
    result_df = page_search(py_tweets,query,tweet_filename,raw_filename) 
    
    logger.debug(f'search result df: \n{result_df}')
    
    # merge dataframes - complete table of search results collected and written out to csv file in code block below
    search_range_results_df = search_range_results_df.append(result_df[0], ignore_index=True)
    search_tweets_text_meta_df = search_tweets_text_meta_df.append(result_df[1], ignore_index=True)
    

In [ ]:
search_range_results_df.head()

In [ ]:
search_tweets_text_meta_df.head()

In [ ]:
sorted_search_df = search_range_results_df.sort_values(by=['month','day_of_month','day_of_week'], ascending=True)
sorted_search_df.head(20)

In [ ]:
logger.info(f'search_range_results_df shape: {search_range_results_df.shape}')

In [ ]:
logger.info(f'x-rate-limit-remaining: [{py_tweets.get_lastfunction_header("x-rate-limit-remaining")}]')
logger.info(f'home_timeline: {py_tweets.get_home_timeline()}')

## Save Full DataFrame of search results to csv

In [ ]:
outputPath = f'{dataDir}/{nfl_type}/{search_on}/v{search_iteration}'
search_range_results_df.to_csv(f'{outputPath}/search_result_tweet_text_data.csv', index=False)
search_tweets_text_meta_df.to_csv(f'{outputPath}/search_result_tweet_text_meta.csv', index=False)